Importing and Cleaning

This notebook serves as introduction to the Kaggle Competition for Text Normalization. Whose data is available here: https://www.kaggle.com/c/text-normalization-challenge-english-language 

I will be using various packages such as numpy for computations, pandas for data frames, and MatPlotLib for visualizations. 

In [9]:
import sys


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'pandas'

In [ ]:
print("Reading in data...")
tr_filepath = "./en_train.csv"
df_train = pd.read_csv(tr_filepath)
dt_train.columns